In [2]:
import os
import re
import sys
import glob
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multcomp
import matplotlib.pyplot as plt


from importlib import reload
import utils # in this directory

%matplotlib inline

In [3]:
reload(utils) # in case changes were made to utils 

<module 'utils' from '/home/ayaankazerouni/Developer/mutation-testing/analysis/utils.py'>

In [24]:
outerdir = '../data/icse-seet/cs3114/fall2018/p*'
mutation_csvs = glob.glob('{outerdir}/*/pitReports/mutations.csv'.format(outerdir=outerdir))
webcat_path = os.path.join('/home/ayaankazerouni/Developer/sensordata/data/fall-2018/submissions.csv')

In [25]:
pit_mutations = []
columns=['className', 'fullQualifiedClassName', 'mutator', 'methodName', 
         'lineNumber', 'killed', 'killingTest']
for datafile in mutation_csvs:
    try:
        username = os.path.dirname(os.path.dirname(datafile))
        assignment = 'Project {}'.format(os.path.basename(os.path.dirname(username))[1])
        username = os.path.basename(username)
        userdata = pd.read_csv(datafile, names=columns)
        userdata['userName'] = username
        userdata['assignment'] = assignment
        pit_mutations.append(userdata)
        del userdata
    except pd.errors.EmptyDataError:
        pass
pit_mutations = pd.concat(pit_mutations, sort=False)

In [26]:
submissions = utils.getsubmissions(webcat_path=webcat_path, users=pit_mutations.userName.unique()) \
    .reset_index()
submissions.loc[:, 'assignment'] = submissions['assignment'] \
    .apply(lambda a: re.search(r'Project \d', a).group())
submissions = submissions.set_index(['userName', 'assignment'])
mutators = utils.get_mutator_specific_data(pit_mutations=pit_mutations, submissions=submissions)
coverage = utils.all_mutator_data(mutators, 'cov')
mutant_counts = utils.all_mutator_data(mutators, 'num')
# drop projects that are all NaN
coverage = coverage.loc[~coverage.isna().all(axis=1)] 
mutant_counts = mutant_counts.loc[~mutant_counts.isna().all(axis=1)]
del pit_mutations # no need to hold this in memory anymore 

In [27]:
full = utils.get_data_for_subset(mutators, submissions=submissions, subset=utils.pit_full)
full.head()

num       cov      surv       eff       mpl
userName assignment                                                
adamt    Project 1   2969.0  0.716403  0.283597  0.040405  7.018913
         Project 2   3899.0  0.674532  0.325468  0.064025  5.083442
adityak8 Project 1   4525.0  0.820110  0.179890  0.018446  9.752155
         Project 2   5509.0  0.829189  0.170811  0.023440  7.287037
afsarver Project 2   2684.0  0.666542  0.333458  0.057771  5.772043

In [38]:
mutant_counts[utils.pit_deletion].divide(mutant_counts[utils.pit_deletion].sum(axis=1), axis=0).median()

mutator
RemoveConditional         0.385965
VoidMethodCall            0.121368
NonVoidMethodCall         0.394966
ConstructorCall           0.037383
BooleanTrueReturnVals     0.010593
BooleanFalseReturnVals    0.008247
PrimitiveReturns          0.021572
EmptyObjectReturnVals     0.008210
dtype: float64

In [20]:
def forward_selection(data, response):
    """Linear model designed by forward selection.
    Credit: https://planspace.org/20150423-forward_selection_with_statsmodels/
    
    Args:
        data (pd.DataFrame): All possible predictors and response
        response (str): Name of response column in `data`
        candidates (list): Candidate features

    Returns:
        model: an "optimal" fitted statsmodels linear model
               with an intercept
               selected by forward selection
               evaluated by Bayesian Information Criterion (BIC)
    """
    # add features in order of the specified sort_criteria
    candidates = list(data.columns)
    candidates.remove(response)
    remaining = candidates.copy() # keep candidates intact to measure savings
    selected = []
    maxint = sys.maxsize
    current_score, best_new_score = maxint, maxint
    
    original_num = utils.get_data_for_subset(mutators, submissions=submissions, subset=candidates) \
        ['num'].sum()
    
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            features = selected + [candidate]
            formula = '{} ~ {} + 1'.format(response, ' + '.join(features))
            
            model = smf.ols(formula, data).fit()
            bic = model.bic
            r_squared = model.rsquared_adj
            scores_with_candidates.append((bic, candidate, r_squared))
            
        scores_with_candidates.sort()
        best_new_score, best_candidate, r_squared = scores_with_candidates[0]
        if current_score > best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            add_num = utils.get_data_for_subset(mutators, submissions=submissions, 
                                                subset=selected)['num'].sum()
            print('Add {}, {:.2%} of FULL mutants. R^2 = {:.2%}'.format(best_candidate, 
                                                                           add_num / full['num'].sum(), 
                                                                           r_squared))
            current_score = best_new_score

    formula = '{} ~ {} + 1'.format(response, ' + '.join(selected))
    print('Selected {} / {} mutators'.format(len(selected), len(candidates)))
    
    model = smf.ols(formula, data).fit()
    return model, selected, add_num

In [28]:
depvar = 'cov'
response = full[depvar]
print('Candidate operators: {}'.format(utils.pit_deletion))
d = coverage[utils.pit_deletion] \
        .merge(response, right_index=True, left_index=True) \
        .dropna(how='any')
model, subset, num = forward_selection(d, depvar)
print('R^2: {:.2%}'.format(model.rsquared_adj), subset)
model.summary()

Candidate operators: ['RemoveConditional', 'VoidMethodCall', 'NonVoidMethodCall', 'ConstructorCall', 'BooleanTrueReturnVals', 'BooleanFalseReturnVals', 'PrimitiveReturns', 'EmptyObjectReturnVals']
Add RemoveConditional, 6.65% of FULL mutants. R^2 = 94.20%
Add NonVoidMethodCall, 13.51% of FULL mutants. R^2 = 95.03%
Add PrimitiveReturns, 13.89% of FULL mutants. R^2 = 95.53%
Selected 3 / 8 mutators
R^2: 95.53% ['RemoveConditional', 'NonVoidMethodCall', 'PrimitiveReturns']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cov   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     1769.
Date:                Wed, 12 Jun 2019   Prob (F-statistic):          1.15e-165
Time:                        12:13:15   Log-Likelihood:                 520.87
No. Observations:                 249   AIC:                            -1034.
Df Residuals:                     245   BIC:                            -1020.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1275      0.013     10.157      0.000       0.103       0.152
RemoveConditional     0.5120      0.031     16.504      0.000       0.451       0.573
NonVoidMethodCall     0.1911      0.030      6.355      0.000       0.132       0.250
PrimitiveReturns      0.0899      0.017      5.366      0.000       0.057       0.123
==============================================================================
Omnibus:                        3.704   Durbin-Watson:                   1.811
Prob(Omnibus):                  0.157   Jarque-Bera (JB):                3.652
Skew:                           0.178   Prob(JB):                        0.161
Kurtosis:                       3.475   Cond. No.                         38.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
depvar = 'cov'
response = full[depvar]
print('Candidate operators: {}'.format(utils.pit_sufficient))
d = coverage[utils.pit_sufficient] \
        .merge(response, right_index=True, left_index=True) \
        .dropna(how='any')
model, subset, num = forward_selection(d, depvar)
print('R^2: {:.2%}'.format(model.rsquared_adj), subset)
model.summary()

Candidate operators: ['ABS', 'ROR', 'AOD', 'UOI']
Add ROR, 12.68% of FULL mutants. R^2 = 94.27%
Add UOI, 36.95% of FULL mutants. R^2 = 97.51%
Add AOD, 40.12% of FULL mutants. R^2 = 98.17%
Add ABS, 46.19% of FULL mutants. R^2 = 98.46%
Selected 4 / 4 mutators
R^2: 98.46% ['ROR', 'UOI', 'AOD', 'ABS']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cov   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     4414.
Date:                Wed, 12 Jun 2019   Prob (F-statistic):          5.67e-246
Time:                        12:13:25   Log-Likelihood:                 726.33
No. Observations:                 277   AIC:                            -1443.
Df Residuals:                     272   BIC:                            -1425.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0273      0.008      3.257      0.001       0.011       0.044
ROR            0.4482      0.015     30.905      0.000       0.420       0.477
UOI            0.2903      0.027     10.796      0.000       0.237       0.343
AOD            0.1294      0.012     10.488      0.000       0.105       0.154
ABS            0.1469      0.020      7.210      0.000       0.107       0.187
==============================================================================
Omnibus:                       13.066   Durbin-Watson:                   1.929
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               13.801
Skew:                          -0.482   Prob(JB):                      0.00101
Kurtosis:                       3.518   Cond. No.                         52.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
depvar = 'cov'
response = full[depvar]
print('Candidate operators: {}'.format(utils.pit_full))
d = coverage[utils.pit_full] \
        .merge(response, right_index=True, left_index=True) \
        .dropna(how='any')
model, subset, num = forward_selection(d, depvar)
print('R^2: {:.2%}'.format(model.rsquared_adj), subset)
model.summary()

Candidate operators: ['ABS', 'AOD', 'AOR', 'BooleanFalseReturnVals', 'BooleanTrueReturnVals', 'CRCR', 'ConditionalsBoundary', 'ConstructorCall', 'EmptyObjectReturnVals', 'Increments', 'InlineConstant', 'Math', 'NegateConditionals', 'NonVoidMethodCall', 'PrimitiveReturns', 'ROR', 'RemoveConditional', 'ReturnVals', 'UOI', 'VoidMethodCall']
Add ROR, 12.68% of FULL mutants. R^2 = 94.33%
Add CRCR, 31.45% of FULL mutants. R^2 = 97.59%
Add UOI, 55.72% of FULL mutants. R^2 = 99.09%
Add NonVoidMethodCall, 62.58% of FULL mutants. R^2 = 99.39%
Add AOR, 68.91% of FULL mutants. R^2 = 99.65%
Add ABS, 74.98% of FULL mutants. R^2 = 99.71%
Add RemoveConditional, 81.63% of FULL mutants. R^2 = 99.77%
Add ReturnVals, 83.00% of FULL mutants. R^2 = 99.78%
Add InlineConstant, 86.88% of FULL mutants. R^2 = 99.79%
Selected 9 / 20 mutators
R^2: 99.79% ['ROR', 'CRCR', 'UOI', 'NonVoidMethodCall', 'AOR', 'ABS', 'RemoveConditional', 'ReturnVals', 'InlineConstant']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cov   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.293e+04
Date:                Wed, 12 Jun 2019   Prob (F-statistic):          9.44e-316
Time:                        12:13:32   Log-Likelihood:                 903.00
No. Observations:                 249   AIC:                            -1786.
Df Residuals:                     239   BIC:                            -1751.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0027      0.004     -0.617      0.538      -0.011       0.006
ROR                   0.1592      0.014     11.384      0.000       0.132       0.187
CRCR                  0.2077      0.015     13.782      0.000       0.178       0.237
UOI                   0.2259      0.011     21.135      0.000       0.205       0.247
NonVoidMethodCall     0.0931      0.007     13.320      0.000       0.079       0.107
AOR                   0.0915      0.006     14.602      0.000       0.079       0.104
ABS                   0.0786      0.009      8.939      0.000       0.061       0.096
RemoveConditional     0.0933      0.013      7.134      0.000       0.068       0.119
ReturnVals            0.0233      0.006      4.040      0.000       0.012       0.035
InlineConstant        0.0330      0.012      2.818      0.005       0.010       0.056
==============================================================================
Omnibus:                       11.679   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.412
Skew:                           0.330   Prob(JB):                     0.000273
Kurtosis:                       4.070   Cond. No.                         115.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# Display summary infromation about arbitrary subsets
def incsubset_info(subset, name, depvar, runningtime_path=None, outfile=None):
    model = smf.ols('{} ~ {} + 1'.format(depvar, ' + '.join(subset)), data=d).fit()
    df = utils.get_data_for_subset(mutators, subset=subset, submissions=submissions)
    df['mpl'] = df['mpl'] * 1000
    df = df.merge(right=joined['full_cov'], right_index=True, left_index=True)
    print('Subset = ', subset)
    mpl_saving_del = (df['mpl'] / (joined['deletion_mpl'] * 1000)).mean()
    rt_saving_del = (df['runningtime'] / (joined['deletion_runningtime'])).mean()
    mpl_saving_full = (df['mpl'] / (joined['full_mpl'] * 1000)).mean()
    print('MPL time = {:.2f}'.format(df['mpl'].mean()))
    print('\t{:.2%} of del, {:.2%} of full'.format(mpl_saving_del, mpl_saving_full))
    if runningtime_path is not None:
        df['runningtime'] = utils.get_running_time(resultfile=runningtime_path)
        rt_saving_full = (df['runningtime'] / (joined['full_runningtime'])).mean()
        print('Running time = {:.2f}'.format(df['runningtime'].mean()))
        print('\t{:.2%} of del, {:.2%} of full'.format(rt_saving_del, rt_saving_full))

    ax = sns.regplot(x='cov', y='full_cov', data=df)
    ax.set(xlabel='Mutation Coverage Under {}'.format(name), 
           ylabel='Mutation Coverage Under the Full PIT Set',
           title='$R^2 = {:.2f}$'.format(model.rsquared_adj))
    sns.despine()
    if outfile:
        plt.tight_layout()
        ax.get_figure().savefig(outfile)

In [ ]:
def concat_incsubset_data(subsetlist):
    prev = None
    for i in range(1, len(subsetlist) + 1):
        subset_data = utils.get_data_for_subset(mutators, subset=subsetlist[:i], submissions=submissions)
        subset_data = subset_data[['cov', 'mpl']]
        subset_data['mpl'] = subset_data['mpl']
        subset_data['subset'] = 'Subset {}'.format('I' * i)
        if prev is None:
            prev = subset_data
        else:
            prev = pd.concat([prev, subset_data])
    return prev

In [ ]:
# Prepare scatterplots
mpl = utils.factorisedsubsets(df=joined, dv='mpl')
cov = utils.factorisedsubsets(df=joined, dv='cov')
rt = utils.factorisedsubsets(df=joined, dv='runningtime')
df = mpl.reset_index() \
        .merge(right=cov.reset_index(), on=['userName', 'subset'],
               right_index=True, left_index=True)

# change this line to filter in/out specific "main subsets"
df = df[df['subset'].isin(['Deletion', 'Sufficient', 'Full'])]

# change this line to toggle the subsets found by forward selection  
# df = pd.concat([df, concat_incsubset_data(subset)], sort=False)

# subsets have the same colours/markers across scatterplots
cp = sns.color_palette()
subset_palette = {'Deletion': cp[0],
                  'Sufficient': cp[1],
                  'Full': cp[2],
                  'Subset I': cp[3],
                  'Subset II': cp[4],
                  'Subset III': cp[8]
}
subset_markers = {'Deletion': 'o',
                  'Sufficient': 'X',
                  'Full': 's',
                  'Subset I': 'P',
                  'Subset II': 'D',
                  'Subset III': '^' 
}

df['subset'] = pd.Categorical(df['subset'], ['Deletion', 
#                                              'Subset I',
                                             'Sufficient',
#                                              'Subset II',
                                             'Full',])
#                                              'Subset III'])

df['mpl'] = df['mpl'] * 1000
ax = sns.scatterplot(x='mpl', y='cov', style='subset', s=50, hue='subset', 
                     data=df, markers=subset_markers, palette=subset_palette)
ax.set(xlabel='# Mutants per KSLoC', ylabel='Mutation Coverage')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles[1:], labels=labels[1:], loc='lower center', ncol=4, bbox_to_anchor=(0.5, 1))
sns.despine()
plt.tight_layout()
# ax.get_figure().savefig('/home/ayaankazerouni/Desktop/main-subsets.eps')

In [37]:
running_time = utils.get_running_time(resultfile='../data/icse-seet/cs3114/fall2016/p2/mutation-results.ndjson')